In [ ]:
#Imports
import pandas as pd
import plotly.graph_objects as go

In [ ]:
#Data Loading
df = pd.read_csv("crime_data_export.csv")
df.columns
df_2024 = df[df['YEAR'] == 2024]

Index(['OBJECTID', 'RowNumber', 'NIBRS_Code', 'NIBRS_Category',
       'Crime_Description', 'Attempted_Completed', 'MONTH', 'DAY', 'YEAR',
       'Disposition_Description', 'DIBRS_Number', 'District',
       'Offense_Category', 'Neighborhood', 'Weapon_Description',
       'Victim_Injury_Description', 'Priority_Board', 'ORC_Description',
       'Booking_Number', 'Relationship_To_Victim', 'ORC_Number', 'Victim_Age',
       'Victim_Race', 'Victim_Sex', 'Victim_Eth', 'Victim_Type', 'ORC_Part',
       'Commit_Date', 'commit_date_local'],
      dtype='object')

## Neighborhood Insights
Looks like Old North, Westwood, Nort Riverdale, Burkhardt, and Eastern Hills, see the most documented crime. 

In [ ]:

neighborhood_crime = df_2024.groupby(by=["Neighborhood"]).size().to_frame(name='Crime_Count')
neighborhood_crime = neighborhood_crime.sort_values(by="Crime_Count", ascending=False)
top_15_neighborhoods = neighborhood_crime.head(15)
top_15_neighborhoods

,Crime_Count
Neighborhood,
WESTWOOD,602
NORTH RIVERDALE,598
OLD NORTH DAYTON,580
BURKHARDT,510
BELMONT,481
LINDEN HEIGHTS,449
EASTERN HILLS,442
WALNUT HILLS,434
DOWNTOWN,426


## Crime Insights
Look and see what crimes are most common

In [ ]:
types_of_crime = df_2024.groupby(by=["NIBRS_Category"]).size().to_frame(name='Crime_Count')
types_of_crime = types_of_crime.sort_values(by="Crime_Count", ascending=False)
top_10_crimes = types_of_crime.head(10)
top_10_crimes


,Crime_Count
NIBRS_Category,
ASSAULT OFFENSES,5369
LARCENY/THEFT OFFENSES,2336
MOTOR VEHICLE THEFT,1756
DESTRUCTION/DAMAGE/VANDALISM OF PROPERTY,1526
BURGLARY/BREAKING AND ENTERING,1040
ALL OTHER OFFENSES,810
ROBBERY,328
"SEX OFFENSES, FORCIBLE",248
"STOLEN PROPERTY OFFENSES (RECEIVING,ETC)",159


## Combine into a Matrix
Combining into a matrix, comparing top 15 neighborhoods for crime, and types of crime. 

In [ ]:

# Create a pivot table showing crime counts by neighborhood and crime type
crime_matrix = df[df['Neighborhood'].isin(top_15_neighborhoods.index) & 
                  df['NIBRS_Category'].isin(top_10_crimes.index)].groupby(
    ['Neighborhood', 'NIBRS_Category']).size().unstack(fill_value=0)

# Reindex to match the order of top 15 neighborhoods and top 10 crimes
crime_matrix = crime_matrix.reindex(index=top_15_neighborhoods.index, 
                                     columns=top_10_crimes.index, 
                                     fill_value=0)

# Add total column for verification
crime_matrix['Total'] = crime_matrix.sum(axis=1)

crime_matrix

fig = go.Figure(data=go.Heatmap(
    z=crime_matrix.drop(columns=['Total']).values,
    x=crime_matrix.columns[:-1],
    y=crime_matrix.index,
    colorscale='reds',
    text=crime_matrix.drop(columns=['Total']).values,
        texttemplate='%{text}',
        textfont={"size": 10},
    colorbar=dict(title='Number of Crimes'),

))

fig.update_layout(
    title='Heatmap of Crime Types by Neighborhood in Dayton, OH (2024)',
    xaxis_title='Type of Crime',
    yaxis_title='Neighborhood',
    height=600,
    width=800,
)
fig.show()
